In [23]:
import sys
sys.path.append('../')

import numpy as np
import pickle
import logging

import utils.download_db as ddb
from utils.global_config import CONFIG

In [24]:
logging.basicConfig(
    filename='run-logs.log', 
    encoding='utf-8', 
    format='%(asctime)s %(levelname)s: %(message)s',
    level=logging.DEBUG, 
    filemode='w'
)

# Get model

In [25]:
model_filename = r'../models/XGBClassifier.pickle'
with open(model_filename, 'rb') as bin_file:
    model = pickle.load(bin_file)

# Download database №1

In [26]:
url = "https://physionet.org/static/published-projects/afdb/mit-bih-atrial-fibrillation-database-1.0.0.zip"
name = "MIT-BIH-AtrialFibrillation"

db_path = ddb.get_db(url, name, "../../analyse/data/")

signals = ddb.get_signals(db_path, reload=False)

In [27]:
import pandas as pd

windows = []
classification = []
for sig in signals:
    for window in sig.windows:
        metrics, has_defect = window.get_data()
        windows.append(metrics)
        classification.append(has_defect)

windows_pd = pd.DataFrame(windows)
classification_pd = pd.DataFrame(classification)

In [28]:
windows_pd

,median,mean,variance,mean_abs,max,min,sum,AAA,AAB,AAC,...,BCC,CAA,CAB,CAC,CBA,CBB,CBC,CCA,CCB,CCC
0,-0.002326,0.055419,0.099190,0.222344,0.923729,-0.420833,1.662562,4,0,2,...,0,2,0,0,1,4,0,0,0,0
1,-0.004695,0.038917,0.093349,0.207399,0.923729,-0.420833,1.167500,5,0,2,...,0,2,0,0,1,4,0,0,0,0
2,-0.004695,0.040613,0.091894,0.205703,0.923729,-0.420833,1.218393,5,0,3,...,0,2,0,0,1,3,0,0,0,0
3,-0.004695,0.038467,0.091732,0.203557,0.923729,-0.420833,1.154017,5,0,3,...,0,2,0,0,1,3,0,0,0,0
4,-0.004785,0.015771,0.082734,0.188534,0.923729,-0.420833,0.473120,5,0,3,...,0,2,0,1,1,3,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2294577,-0.224906,2.857417,155.126511,3.605519,61.406250,-1.000000,68.578016,0,0,0,...,2,0,1,1,0,2,2,1,2,5
2294578,-0.236220,2.727775,149.324823,3.476644,61.406250,-1.000000,68.194373,0,0,0,...,2,0,1,1,0,2,2,1,2,5
2294579,-0.213592,5.590630,328.090512,6.307107,71.166466,-1.000000,139.765746,0,0,0,...,2,0,1,0,0,2,2,1,2,5
2294580,-0.236220,5.551042,328.571580,6.346694,71.166466,-1.000000,138.776056,0,0,0,...,2,0,1,0,0,2,3,1,2,5


In [29]:
classification_pd

,0
0,False
1,False
2,False
3,False
4,False
...,...
2294577,True
2294578,True
2294579,True
2294580,True


# Tests

In [31]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score


results = pd.DataFrame()

for i in range(100):
    _, X_test, _, y_test = train_test_split(windows_pd, classification_pd, train_size=0.9, random_state=i)
    y_pred = model.predict(X_test)
    f1 = f1_score(y_test, y_pred)
    result = model.score(X_test, y_test)
    df = pd.DataFrame({"score" : result, "error" : 1 - result, "f1 norm" : f1}, index=[i])
    results = pd.concat([results, df])


In [32]:
results

,score,error,f1 norm
0,0.994738,0.005262,0.971831
1,0.994680,0.005320,0.972002
2,0.994557,0.005443,0.971998
3,0.994672,0.005328,0.971725
4,0.994852,0.005148,0.972142
...,...,...,...
95,0.994558,0.005442,0.971667
96,0.994665,0.005335,0.971674
97,0.994656,0.005344,0.971618
98,0.994747,0.005253,0.971820


In [33]:
print("mean score:\t", np.mean(results["score"]))
print("min score:\t", np.min(results["score"]))
print("max score:\t", np.max(results["score"]))
print("mean f1 norm:\t", np.max(results["f1 norm"]))

mean score:	 0.9947382043820501
min score:	 0.9944240244192761
max score:	 0.9949863977996444
mean f1 norm:	 0.9725540001680939


# Download database №2

In [34]:
url = "https://physionet.org/static/published-projects/vfdb/mit-bih-malignant-ventricular-ectopy-database-1.0.0.zip"
name = "MIT-BIH-Malignant-Ventricular-Ectopy"

db_path = ddb.get_db(url, name, "../../analyse/data/")

signals = ddb.get_signals(db_path, reload=False)

In [35]:
import pandas as pd

windows = []
classification = []
for sig in signals:
    for window in sig.windows:
        metrics, has_defect = window.get_data()
        windows.append(metrics)
        classification.append(has_defect)

windows_pd = pd.DataFrame(windows)
classification_pd = pd.DataFrame(classification)

In [36]:
windows_pd

,median,mean,variance,mean_abs,max,min,sum,AAA,AAB,AAC,...,BCC,CAA,CAB,CAC,CBA,CBB,CBC,CCA,CCB,CCC
0,-0.042558,0.110940,0.314108,0.389761,1.261905,-0.413043,3.328204,3,0,2,...,3,3,1,0,0,0,6,0,3,0
1,-0.021796,0.122727,0.306837,0.377974,1.261905,-0.413043,3.681810,4,0,2,...,3,3,1,0,0,0,5,0,3,0
2,-0.021796,0.082418,0.262118,0.337665,1.259259,-0.413043,2.472537,5,0,2,...,3,3,1,0,0,0,5,0,3,0
3,-0.021796,0.090877,0.257901,0.329206,1.259259,-0.413043,2.726296,6,0,2,...,3,2,1,0,0,0,5,0,3,0
4,-0.032246,0.076556,0.266664,0.343527,1.259259,-0.437037,2.296667,5,0,3,...,3,2,1,0,0,0,5,0,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137633,0.029740,0.144267,0.373776,0.458281,1.949367,-0.658747,4.328006,0,0,0,...,3,0,0,0,0,4,4,0,5,0
137634,-0.048210,0.124958,0.378138,0.456837,1.949367,-0.658747,3.748738,0,0,0,...,2,0,0,0,0,4,5,0,5,0
137635,0.029740,0.153716,0.374756,0.462206,1.949367,-0.658747,4.611483,0,0,0,...,2,0,0,0,0,4,5,0,4,0
137636,0.029740,0.152094,0.376591,0.463828,1.949367,-0.658747,4.562824,0,0,0,...,2,0,0,0,0,4,5,0,4,0


In [37]:
classification_pd

,0
0,False
1,False
2,False
3,False
4,False
...,...
137633,True
137634,True
137635,True
137636,True


# Tests

In [38]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score


results = pd.DataFrame()

for i in range(100):
    _, X_test, _, y_test = train_test_split(windows_pd, classification_pd, train_size=0.9, random_state=i)
    y_pred = model.predict(X_test)
    f1 = f1_score(y_test, y_pred)
    result = model.score(X_test, y_test)
    df = pd.DataFrame({"score" : result, "error" : 1 - result, "f1 norm" : f1}, index=[i])
    results = pd.concat([results, df])


In [39]:
results

,score,error,f1 norm
0,0.852510,0.147490,0.818492
1,0.857414,0.142586,0.825419
2,0.855297,0.144703,0.816775
3,0.856443,0.143557,0.819519
4,0.861201,0.138799,0.822790
...,...,...,...
95,0.852380,0.147620,0.816847
96,0.855379,0.144621,0.821362
97,0.859072,0.140928,0.823925
98,0.851478,0.148522,0.817229


In [40]:
print("mean score:\t", np.mean(results["score"]))
print("min score:\t", np.min(results["score"]))
print("max score:\t", np.max(results["score"]))
print("mean f1 norm:\t", np.max(results["f1 norm"]))

mean score:	 0.8542222350922373
min score:	 0.8474071445514679
max score:	 0.8612011425795518
mean f1 norm:	 0.8254194538874876
